In [1]:
# load libraries
import os
import subprocess

In [2]:
class _Files:
    """Commentary"""
    
    def __init__(self, script, sspath, rutass, lineno, marker):
        """Variables
        ------------------------
        nums    : file counter
        eexe    : name of the script
        path    : path to the script
        ruts    : path to the file
        linn    : line number of the \
                  file call on script
        ifin    : input or ouput file
        """
        self.nums = 1
        self.eexe = [script]
        self.path = [sspath]
        self.ruta = [rutass]
        self.linn = [lineno]
        self.ifin = [True if marker == "in" else False]
        
        
    def add_data(self, script, sspath, rutass, lineno, marker):
        # increment 
        self.nums += 1
        # fill list
        self.eexe.append(script)
        self.path.append(sspath)
        self.ruta.append(rutass)
        self.linn.append(lineno)
        self.ifin.append(True if marker == "in" else False)

    def connections(self):
        # map inputs to outputs
        return 0

def add_to_dict(data, scripts, spathss, infiles, rutasss, linenos, markers):
    """Inputs
    ------------------------
    data    : dict
    scripts : name of the R scripts
    spathss : the paths to those scripts
    infiles : all the files loaded/saved
    rutasss : the paths to those files
    linenos : the line number where the files is called
    markers : indicate if the files are inputs or outputs
    """
        
    for c,v in enumerate(infiles):
        if data.get(v, None) is None: 
            # create object
            data[v] = _Files(scripts[c], spathss[c], rutasss[c], linenos[c], markers[c])
        else: 
            # append to object
            data[v].add_data(scripts[c], spathss[c], rutasss[c], linenos[c], markers[c])

In [3]:
def read_dir(dirpath):
    """Comments"""
    
    # bash command
    greppp = "grep -rnw"
    # look for input file
    patter = "-e 'load('"
    # look for output files
    patteo = "-e 'save(.*file='"
    
    # Save output in tmp lists
    loadd = subprocess.run(greppp + " " + dirpath + " " + patter, 
                       shell=True, universal_newlines=True, 
                       stdout=subprocess.PIPE)
    
    saved = subprocess.run(greppp + " " + dirpath + " " + patteo, shell=True, 
                       universal_newlines=True, 
                       stdout=subprocess.PIPE)
    
    # counter
    k = 0
    # loadd size
    l = len(loadd.stdout.splitlines())
    
    # data dict 
    # of class objects
    data = {}
    
    #-----------------------------------------------------------------
    # ../Mapping_aa.R:20:load("../T26_CEL_Subject.Rdata")
    # '../' - 'Mapping_aa.R' - '20' - '../' - 'T26_CEL_Subject.Rdata'
    # path  -      base      - lino -  ruta -          file
    #-----------------------------------------------------------------
    
    # lists
    path = []
    base = []
    file = []
    ruta = []
    lino = []
    mark = []
    
    # READ ALL SAVE IN DICT
    for line in (loadd.stdout + saved.stdout).splitlines():
        
        # increase counter
        k += 1
        
        # split line 
        array = str(line).split(":")
        
        # line number
        lino.append(array[1])
        
        # get path and script name
        path.append(os.path.dirname(array[0]))
        base.append(os.path.basename(array[0]))
        
        # get name of file
        temp = array[2].split("\"")
        ruta.append(os.path.dirname(temp[-2]))
        file.append(os.path.basename(temp[-2]))
        
        # inputs
        mark.append("in" if k<=l else "out")
    
    # add to dict
    add_to_dict(data, base, path, file, ruta, lino, mark)
    
    # end
    return data

In [6]:
def save_file(filepath, data):
    """Save the results in a csv format file"""
    
    try:
        # create file
        file = open(filepath, "x")
    
    except IOError:
        print("\nWarning!! Output File already exist")
        print("File will be appended\n")
        file = open(filepath, "a")
    
    finally:
        # write file headder
        file.write("File_name, File_paths, Input, Script_name, Script_paths\n")
        # print data records
        for key, value in data.items():
            for i in range(value.nums): 
                file.write("%i, %s,%s,%r,%s,%s\n" % (i, key, value.ruta[i], 
                                                      value.ifin[i], 
                                                      value.eexe[i], 
                                                      value.path[i]))
        # close file
        file.close()

In [7]:
if __name__ == "__main__":
    
    dirpath = "PAPER_DATA/Code/Macrophage"
    data = read_dir(dirpath)
    save_file('./example.csv',data)


Warning!! Output File already exist
File will be appended

